In [109]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import configparser
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import os

In [110]:
train_data = pd.read_csv('../data/processed/feature_for_models.csv')
test_data = pd.read_csv('../data/processed/test_dataset.csv')

In [111]:
with open('../artifacts/pipeline.pkl','rb') as f:
    loans_model = pickle.load(f)

In [112]:
loans_model

Pipeline(steps=[('freq_var_encoding',
                 CountFrequencyEncoder(variables=['Código Moneda',
                                                  'Status Mora', 'Nuevo'])),
                ('lab_var_encoding',
                 OrdinalEncoder(variables=['Brecha_Plazo'])),
                ('capping_var',
                 Winsorizer(capping_method='iqr', fold=1.5, tail='both',
                            variables=['Saldo$', 'PE_30.04.2024',
                                       'PlazoRestante(Dias)',
                                       'Reserva Especifica$', 'Ab_24',
                                       'Tasa_Propuesta',
                                       'IntAnual_TasaPropuesta', 'FTP',
                                       'TIR'])),
                ('feature_scaling', StandardScaler())])

In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV #para hiper-parámetros

# metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

#### Agregar lectura de target con el archivo de configuración

In [114]:
x_features = train_data.drop(labels=['SegmentoComercial'], axis = 1)
y_target = train_data['SegmentoComercial']

x_features_test = test_data.drop(labels=['SegmentoComercial'], axis = 1)
y_target_test = test_data['SegmentoComercial']

In [115]:
# Configuración 1 Random Forest
rf_model_1 = RandomForestClassifier(
    n_estimators=100,           # Número de árboles en el bosque
    max_depth=10,               # Profundidad máxima de los árboles
    min_samples_split=5,        # Número mínimo de muestras requeridas para dividir un nodo
    min_samples_leaf=3,         # Número mínimo de muestras en una hoja
    max_features='sqrt',        # Número de características a considerar al buscar la mejor división
    random_state=42             # Fijar la semilla para reproducibilidad
)

# Entrenar el modelo
rf_model_1.fit(x_features, y_target)

# Predecir
y_preds_1 = rf_model_1.predict(x_features_test)

# Calcular la precisión
rf_acc_1 = accuracy_score(y_target_test, y_preds_1)

# Mostrar la precisión
print(f"Accuracy of Random Forest Model 1: {rf_acc_1:.4f}")

# Configuración 2 Random Forest
rf_model_2 = RandomForestClassifier(
    n_estimators=200,           # Número de árboles en el bosque
    max_depth=5,               # Profundidad máxima de los árboles
    min_samples_split=8,        # Número mínimo de muestras requeridas para dividir un nodo
    min_samples_leaf=3,         # Número mínimo de muestras en una hoja
    max_features='sqrt',        # Número de características a considerar al buscar la mejor división
    random_state=2024             # Fijar la semilla para reproducibilidad
)

# Entrenar el modelo
rf_model_2.fit(x_features, y_target)

# Predecir
y_preds_2 = rf_model_2.predict(x_features_test)

# Calcular la precisión
rf_acc_2 = accuracy_score(y_target_test, y_preds_2)

# Mostrar la precisión
print(f"Accuracy of Random Forest Model 2: {rf_acc_2:.4f}")


# Configuración 3 Random Forest
rf_model_3 = RandomForestClassifier(
    n_estimators=300,           # Número de árboles en el bosque
    max_depth=10,               # Profundidad máxima de los árboles
    min_samples_split=10,        # Número mínimo de muestras requeridas para dividir un nodo
    min_samples_leaf=3,         # Número mínimo de muestras en una hoja
    max_features='sqrt',        # Número de características a considerar al buscar la mejor división
    random_state=40             # Fijar la semilla para reproducibilidad
)

# Entrenar el modelo
rf_model_3.fit(x_features, y_target)

# Predecir
y_preds_3 = rf_model_3.predict(x_features_test)

# Calcular la precisión
rf_acc_3 = accuracy_score(y_target_test, y_preds_3)

# Mostrar la precisión
print(f"Accuracy of Random Forest Model 3: {rf_acc_3:.4f}")

Accuracy of Random Forest Model 1: 0.9823
Accuracy of Random Forest Model 2: 0.9787
Accuracy of Random Forest Model 3: 0.9823


In [116]:
# Configuración 1 Regresión Logistica
lr_model_1 = LogisticRegression(
    penalty='l2',              # Regularización L2
    C=1.0,                     # Inverso de la fuerza de regularización
    solver='liblinear',        # Solución para pequeños datasets
    max_iter=100,              # Iteraciones máximas
    random_state=42
)
lr_model_1.fit(x_features, y_target)
lr_preds_1 = lr_model_1.predict(x_features_test)
lr_acc_1 = accuracy_score(y_target_test, lr_preds_1)

# Mostrar la precisión
print(f"Accuracy of Logistic Regression 1: {lr_acc_1:.4f}")

# Configuración 2
lr_model_2 = LogisticRegression(
    penalty='l1',              # Regularización L1
    C=0.5,                     # Fuerza de regularización más alta
    solver='saga',             # Compatible con L1
    max_iter=200,
    random_state=42
)
lr_model_2.fit(x_features, y_target)
lr_preds_2 = lr_model_2.predict(x_features_test)
lr_acc_2 = accuracy_score(y_target_test, lr_preds_2)

# Mostrar la precisión
print(f"Accuracy of Logistic Regression 2: {lr_acc_2:.4f}")

# Configuración 3
lr_model_3 = LogisticRegression(
    penalty='elasticnet',      # ElasticNet combina L1 y L2
    C=0.1,
    solver='saga',
    l1_ratio=0.5,              # Proporción de L1 en ElasticNet
    max_iter=300,
    random_state=42
)
lr_model_3.fit(x_features, y_target)
lr_preds_3 = lr_model_3.predict(x_features_test)
lr_acc_3 = accuracy_score(y_target_test, lr_preds_3)

# Mostrar la precisión
print(f"Accuracy of Logistic Regression 3: {lr_acc_3:.4f}")


Accuracy of Logistic Regression 1: 0.9492


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy of Logistic Regression 2: 0.9468
Accuracy of Logistic Regression 3: 0.9480


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [117]:
# Configuración 1 SVC
svm_model_1 = SVC(
    kernel='linear',           # Kernel lineal
    C=1.0,                     # Regularización estándar
    probability=True,          # Probabilidades habilitadas
    random_state=42
)
svm_model_1.fit(x_features, y_target)
svm_preds_1 = svm_model_1.predict(x_features_test)
svm_acc_1 = accuracy_score(y_target_test, svm_preds_1)

# Mostrar la precisión
print(f"Accuracy of SVM 1: {svm_acc_1:.4f}")

# Configuración 2 SVC
svm_model_2 = SVC(
    kernel='rbf',              # Kernel Gaussiano
    C=0.5,
    gamma='scale',             # Parámetro gamma automático
    random_state=42
)
svm_model_2.fit(x_features, y_target)
svm_preds_2 = svm_model_2.predict(x_features_test)
svm_acc_2 = accuracy_score(y_target_test, svm_preds_2)

# Mostrar la precisión
print(f"Accuracy of SVM 2: {svm_acc_2:.4f}")

# Configuración 3 SVC
svm_model_3 = SVC(
    kernel='poly',             # Kernel polinomial
    degree=3,                  # Grado del polinomio
    C=1.5,
    probability=True,
    random_state=42
)
svm_model_3.fit(x_features, y_target)
svm_preds_3 = svm_model_3.predict(x_features_test)
svm_acc_3 = accuracy_score(y_target_test, svm_preds_3)

# Mostrar la precisión
print(f"Accuracy of SVM 3: {svm_acc_3:.4f}")

Accuracy of SVM 1: 0.9775
Accuracy of SVM 2: 0.9492
Accuracy of SVM 3: 0.9551


In [118]:
# Configuración 1 AdaBoost Classifier
adaboost_model_1 = AdaBoostClassifier(
    n_estimators=50,           # Número de estimadores
    learning_rate=1.0,         # Tasa de aprendizaje estándar
    random_state=42
)
adaboost_model_1.fit(x_features, y_target)
adaboost_preds_1 = adaboost_model_1.predict(x_features_test)
adaboost_acc_1 = accuracy_score(y_target_test, adaboost_preds_1)

# Mostrar la precisión
print(f"Accuracy of AdaBoost 1: {adaboost_acc_1:.4f}")

# Configuración 2 AdaBoost Classifier
adaboost_model_2 = AdaBoostClassifier(
    n_estimators=100,
    learning_rate=0.5,         # Tasa de aprendizaje más baja
    random_state=42
)
adaboost_model_2.fit(x_features, y_target)
adaboost_preds_2 = adaboost_model_2.predict(x_features_test)
adaboost_acc_2 = accuracy_score(y_target_test, adaboost_preds_2)

# Mostrar la precisión
print(f"Accuracy of AdaBoost 2: {adaboost_acc_2:.4f}")

# Configuración 3 AdaBoost Classifier
adaboost_model_3 = AdaBoostClassifier(
    n_estimators=200,
    learning_rate=0.1,         # Tasa de aprendizaje muy baja
    random_state=42
)
adaboost_model_3.fit(x_features, y_target)
adaboost_preds_3 = adaboost_model_3.predict(x_features_test)
adaboost_acc_3 = accuracy_score(y_target_test, adaboost_preds_3)

# Mostrar la precisión
print(f"Accuracy of AdaBoost 3: {adaboost_acc_3:.4f}")


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy of AdaBoost 1: 0.9125


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy of AdaBoost 2: 0.9255


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy of AdaBoost 3: 0.9102


In [119]:
# Configuración 1 XGBoost
xgb_model_1 = xgb.XGBClassifier(
    n_estimators=100,          # Número de árboles
    max_depth=6,               # Profundidad máxima de los árboles
    learning_rate=0.3,         # Tasa de aprendizaje estándar
    random_state=42,
    use_label_encoder=False
)
xgb_model_1.fit(x_features, y_target)
xgb_preds_1 = xgb_model_1.predict(x_features_test)
xgb_acc_1 = accuracy_score(y_target_test, xgb_preds_1)

# Mostrar la precisión
print(f"Accuracy of XGBoost 1: {xgb_acc_1:.4f}")

# Configuración 2 XGBoost
xgb_model_2 = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,             # Submuestreo de datos
    colsample_bytree=0.8,      # Submuestreo de características
    random_state=42,
    use_label_encoder=False
)
xgb_model_2.fit(x_features, y_target)
xgb_preds_2 = xgb_model_2.predict(x_features_test)
xgb_acc_2 = accuracy_score(y_target_test, xgb_preds_2)

# Mostrar la precisión
print(f"Accuracy of XGBoost 2: {xgb_acc_2:.4f}")

# Configuración 3 XGBoost
xgb_model_3 = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.6,
    colsample_bytree=0.6,
    gamma=1.0,                 # Regularización
    random_state=42,
    use_label_encoder=False
)
xgb_model_3.fit(x_features, y_target)
xgb_preds_3 = xgb_model_3.predict(x_features_test)
xgb_acc_3 = accuracy_score(y_target_test, xgb_preds_3)

# Mostrar la precisión
print(f"Accuracy of XGBoost 3: {xgb_acc_3:.4f}")


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:45:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy of XGBoost 1: 0.9799


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:45:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy of XGBoost 2: 0.9835


c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:45:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy of XGBoost 3: 0.9823


In [120]:
# Lista de todos los accuracies
rf_acc = max(rf_acc_1, rf_acc_2, rf_acc_3)
lr_acc = max(lr_acc_1, lr_acc_2, lr_acc_3)
svm_acc = max(svm_acc_1, svm_acc_2, svm_acc_3)
ada_acc = max(adaboost_acc_1, adaboost_acc_2, adaboost_acc_3)
xgb_acc = max(xgb_acc_1, xgb_acc_2, xgb_acc_3)

# Encontrar el mejor accuracy general
modelos = [rf_acc, lr_acc, svm_acc, ada_acc, xgb_acc]
MejorModelo = max(modelos)

if MejorModelo == rf_acc:
    # Determinar la configuración específica de RandomForest
    if rf_acc == rf_acc_1:
        loans_model.steps.append(
            ('modelo_random_forest', RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42))
        )
    elif rf_acc == rf_acc_2:
        loans_model.steps.append(
            ('modelo_random_forest', RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42))
        )
    else:
        loans_model.steps.append(
            ('modelo_random_forest', RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42))
        )

elif MejorModelo == lr_acc:
    # Determinar la configuración específica de LogisticRegression
    if lr_acc == lr_acc_1:
        loans_model.steps.append(
            ('modelo_logistic_regression', LogisticRegression(C=1.0, penalty='l2', solver='lbfgs', random_state=42))
        )
    elif lr_acc == lr_acc_2:
        loans_model.steps.append(
            ('modelo_logistic_regression', LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=42))
        )
    else:
        loans_model.steps.append(
            ('modelo_logistic_regression', LogisticRegression(C=2.0, penalty='l2', solver='lbfgs', random_state=42))
        )

elif MejorModelo == svm_acc:
    # Determinar la configuración específica de SVM
    if svm_acc == svm_acc_1:
        loans_model.steps.append(
            ('modelo_svm', SVC(C=1.0, kernel='linear', random_state=42))
        )
    elif svm_acc == svm_acc_2:
        loans_model.steps.append(
            ('modelo_svm', SVC(C=0.5, kernel='rbf', gamma=0.1, random_state=42))
        )
    else:
        loans_model.steps.append(
            ('modelo_svm', SVC(C=2.0, kernel='poly', degree=3, random_state=42))
        )

elif MejorModelo == ada_acc:
    # Determinar la configuración específica de AdaBoost
    if ada_acc == adaboost_acc_1:
        loans_model.steps.append(
            ('modelo_ada_boost', AdaBoostClassifier(n_estimators=50, learning_rate=1.0, random_state=42))
        )
    elif ada_acc == adaboost_acc_2:
        loans_model.steps.append(
            ('modelo_ada_boost', AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42))
        )
    else:
        loans_model.steps.append(
            ('modelo_ada_boost', AdaBoostClassifier(n_estimators=150, learning_rate=0.3, random_state=42))
        )

else:
    # Determinar la configuración específica de XGBoost
    if xgb_acc == xgb_acc_1:
        loans_model.steps.append(
            ('modelo_xgb', xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.3, random_state=42, use_label_encoder=False))
        )
    elif xgb_acc == xgb_acc_2:
        loans_model.steps.append(
            ('modelo_xgb', xgb.XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, random_state=42, use_label_encoder=False))
        )
    else:
        loans_model.steps.append(
            ('modelo_xgb', xgb.XGBClassifier(n_estimators=300, max_depth=8, learning_rate=0.05, subsample=0.6, colsample_bytree=0.6, gamma=1.0, random_state=42, use_label_encoder=False))
        )

# Mostrar resultados
print(f"El mejor modelo tiene una precisión de {MejorModelo:.4f}")
print(f"Pipeline final: {loans_model}")


El mejor modelo tiene una precisión de 0.9835
Pipeline final: Pipeline(steps=[('freq_var_encoding',
                 CountFrequencyEncoder(variables=['Código Moneda',
                                                  'Status Mora', 'Nuevo'])),
                ('lab_var_encoding',
                 OrdinalEncoder(variables=['Brecha_Plazo'])),
                ('capping_var',
                 Winsorizer(capping_method='iqr', fold=1.5, tail='both',
                            variables=['Saldo$', 'PE_30.04.2024',
                                       'PlazoRestante(Dias)',
                                       'Reserva Especifica$', 'Ab_24',
                                       'Tasa_Propuesta',
                                       'IntA...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max

In [121]:
loans_model

Pipeline(steps=[('freq_var_encoding',
                 CountFrequencyEncoder(variables=['Código Moneda',
                                                  'Status Mora', 'Nuevo'])),
                ('lab_var_encoding',
                 OrdinalEncoder(variables=['Brecha_Plazo'])),
                ('capping_var',
                 Winsorizer(capping_method='iqr', fold=1.5, tail='both',
                            variables=['Saldo$', 'PE_30.04.2024',
                                       'PlazoRestante(Dias)',
                                       'Reserva Especifica$', 'Ab_24',
                                       'Tasa_Propuesta',
                                       'IntA...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

#### configuración y entrenamiento de modelo final

In [122]:
config = configparser.ConfigParser()
config.read('../pipeline.cfg')
data_path = os.path.abspath("../data/raw/DataLoans_0424.csv")
dataset = pd.read_csv(data_path, delimiter = ";")
dataset['SegmentoComercial'] = label_encoder.fit_transform(dataset['SegmentoComercial'])

In [123]:
drop_vars = list(config.get('GENERAL','VARS_TO_DROP').split(','))
drop_vars.append(config.get('GENERAL','TARGET'))

x_features = dataset.drop(labels = drop_vars, axis = 1)
y_target =dataset[config.get('GENERAL','TARGET')]
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = 0.3, shuffle = True, random_state = 2025)

In [124]:
loans_model.fit(x_train, y_train)

c:\Users\Trull\anaconda3\envs\prod-dev\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:45:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('freq_var_encoding',
                 CountFrequencyEncoder(variables=['Código Moneda',
                                                  'Status Mora', 'Nuevo'])),
                ('lab_var_encoding',
                 OrdinalEncoder(variables=['Brecha_Plazo'])),
                ('capping_var',
                 Winsorizer(capping_method='iqr', fold=1.5, tail='both',
                            variables=['Saldo$', 'PE_30.04.2024',
                                       'PlazoRestante(Dias)',
                                       'Reserva Especifica$', 'Ab_24',
                                       'Tasa_Propuesta',
                                       'IntA...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [125]:
with open('../artifacts/pipeline.pkl','wb') as f:
        pickle.dump(loans_model,f)